In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import explode, split, col, regexp_replace, size
import requests

In [3]:
spark = SparkSession.builder.appName("AnalilisLibro").getOrCreate()

Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/10/14 22:35:34 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
25/10/14 22:35:35 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
25/10/14 22:35:35 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.
25/10/14 22:35:35 WARN Utils: Service 'SparkUI' could not bind on port 4042. Attempting port 4043.


In [4]:
url = "https://www.gutenberg.org/cache/epub/28885/pg28885.txt"

In [5]:
response = requests.get(url)
libro_texto = response.text

In [7]:
with open("libro.txt", "w", encoding="utf-8") as file:
    file.write(libro_texto)

In [8]:
!ls

 AWS_S3			   Untitled4.ipynb   awscliv2.zip
'Analisis de df 1.ipynb'   Untitled5.ipynb   ejercicios_python_desde_cero.ipynb
 Netflix.ipynb		   Untitled6.ipynb   libro.txt
 Untitled.ipynb		   Untitled7.ipynb   netflix_titles.csv
 Untitled1.ipynb	   Untitled8.ipynb   netlflix.ipynb
 Untitled10.ipynb	   Untitled9.ipynb   r_m.csv
 Untitled11.ipynb	   actions-runner    rick_and_morty_characters.csv
 Untitled12.ipynb	   app.py	     titanic.zip
 Untitled2.ipynb	   app_rm.py	     venv
 Untitled3.ipynb	   aws


In [10]:
rdd = spark.sparkContext.textFile("libro.txt")

In [11]:
print(rdd)

libro.txt MapPartitionsRDD[1] at textFile at NativeMethodAccessorImpl.java:0


In [12]:
rdd = rdd.filter(lambda line: line.strip() !="")

In [17]:
capitulos = rdd.filter(lambda line: line.strip().startswith("CHAPTER")).collect()

In [18]:
parrafos = rdd.flatMap(lambda line: line.split("\n\n"))

In [19]:
df_parrafos = parrafos.map(lambda p: (p,)).toDF(["parrafos"])

In [22]:
df_parrafos.show()

+--------------------+
|            parrafos|
+--------------------+
|The Project Guten...|
|This ebook is for...|
|most other parts ...|
|whatsoever. You m...|
|of the Project Gu...|
|at www.gutenberg....|
|you will have to ...|
|before using this...|
|Title: Alice's Ad...|
|Author: Lewis Car...|
|Contributor: Aust...|
|Illustrator: Arth...|
|Release date: May...|
|                M...|
|   Language: English|
|Credits: Produced...|
|        Proofread...|
|        produced ...|
|        Universit...|
|*** START OF THE ...|
+--------------------+
only showing top 20 rows


In [24]:
# Limpiar el texto: eliminar caracteres no alfanuméricos y convertir a minúsculas
df_parrafos = df_parrafos.withColumn("parrafo_limpio", regexp_replace(col("parrafos"), "[^a-zA-Z\s]", "").alias("parrafo_limpio"))
df_parrafos = df_parrafos.withColumn("parrafo_limpio", regexp_replace(col("parrafo_limpio"), "\s+", " ").alias("parrafo_limpio"))

# Dividir los párrafos en oraciones y palabras
df_parrafos = df_parrafos.withColumn("oraciones", split(col("parrafo_limpio"), "\."))
df_parrafos = df_parrafos.withColumn("num_oraciones", size(col("oraciones")))
df_parrafos = df_parrafos.withColumn("palabras", split(col("parrafo_limpio"), " "))
df_parrafos = df_parrafos.withColumn("num_palabras", size(col("palabras")))


<>:2: SyntaxWarning: invalid escape sequence '\s'
<>:3: SyntaxWarning: invalid escape sequence '\s'
<>:6: SyntaxWarning: invalid escape sequence '\.'
<>:2: SyntaxWarning: invalid escape sequence '\s'
<>:3: SyntaxWarning: invalid escape sequence '\s'
<>:6: SyntaxWarning: invalid escape sequence '\.'
/tmp/ipykernel_110053/2205023265.py:2: SyntaxWarning: invalid escape sequence '\s'
  df_parrafos = df_parrafos.withColumn("parrafo_limpio", regexp_replace(col("parrafos"), "[^a-zA-Z\s]", "").alias("parrafo_limpio"))
/tmp/ipykernel_110053/2205023265.py:3: SyntaxWarning: invalid escape sequence '\s'
  df_parrafos = df_parrafos.withColumn("parrafo_limpio", regexp_replace(col("parrafo_limpio"), "\s+", " ").alias("parrafo_limpio"))
/tmp/ipykernel_110053/2205023265.py:6: SyntaxWarning: invalid escape sequence '\.'
  df_parrafos = df_parrafos.withColumn("oraciones", split(col("parrafo_limpio"), "\."))


In [25]:
df_parrafos.show()

+--------------------+--------------------+--------------------+-------------+--------------------+------------+
|            parrafos|      parrafo_limpio|           oraciones|num_oraciones|            palabras|num_palabras|
+--------------------+--------------------+--------------------+-------------+--------------------+------------+
|The Project Guten...|The Project Guten...|[The Project Gute...|            1|[The, Project, Gu...|           9|
|This ebook is for...|This ebook is for...|[This ebook is fo...|            1|[This, ebook, is,...|          14|
|most other parts ...|most other parts ...|[most other parts...|            1|[most, other, par...|          14|
|whatsoever. You m...|whatsoever You ma...|[whatsoever You m...|            1|[whatsoever, You,...|          14|
|of the Project Gu...|of the Project Gu...|[of the Project G...|            1|[of, the, Project...|          11|
|at www.gutenberg....|at wwwgutenbergor...|[at wwwgutenbergo...|            1|[at, wwwgutenberg.

In [26]:
df_parrafos = df_parrafos.withColumn("longitud_promedio_palabra", col("num_palabras") / col("num_oraciones"))


In [27]:
df_parrafos.show()

+--------------------+--------------------+--------------------+-------------+--------------------+------------+-------------------------+
|            parrafos|      parrafo_limpio|           oraciones|num_oraciones|            palabras|num_palabras|longitud_promedio_palabra|
+--------------------+--------------------+--------------------+-------------+--------------------+------------+-------------------------+
|The Project Guten...|The Project Guten...|[The Project Gute...|            1|[The, Project, Gu...|           9|                      9.0|
|This ebook is for...|This ebook is for...|[This ebook is fo...|            1|[This, ebook, is,...|          14|                     14.0|
|most other parts ...|most other parts ...|[most other parts...|            1|[most, other, par...|          14|                     14.0|
|whatsoever. You m...|whatsoever You ma...|[whatsoever You m...|            1|[whatsoever, You,...|          14|                     14.0|
|of the Project Gu...|of th

In [28]:
df_parrafos = df_parrafos.withColumn("vocabulario_unico", size(split(col("parrafo_limpio"), " ")))


In [30]:
df_parrafos.select("parrafos", "num_oraciones", "num_palabras", "longitud_promedio_palabra", "vocabulario_unico").show(truncate=False)

+-----------------------------------------------------------------------------+-------------+------------+-------------------------+-----------------+
|parrafos                                                                     |num_oraciones|num_palabras|longitud_promedio_palabra|vocabulario_unico|
+-----------------------------------------------------------------------------+-------------+------------+-------------------------+-----------------+
|The Project Gutenberg eBook of Alice's Adventures in Wonderland              |1            |9           |9.0                      |9                |
|This ebook is for the use of anyone anywhere in the United States and        |1            |14          |14.0                     |14               |
|most other parts of the world at no cost and with almost no restrictions     |1            |14          |14.0                     |14               |
|whatsoever. You may copy it, give it away or re-use it under the terms       |1            |1

In [31]:
spark.stop()